In [1]:
import pandas as pd
import numpy as np

## Homework

> Note: sometimes your answer doesn't match one of the options exactly.
> That's fine.
> Select the option that's closest to your solution.

### Dataset

In this homework, we will use the lead scoring dataset Bank Marketing dataset. Download it from [here](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv).

Or you can do it with `wget`:

```bash
wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
```

In this dataset our desired target for classification task will be `converted` variable - has the client signed up to the platform or not.

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'
!wget  -P ../data $data

--2025-10-21 20:31:37--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: '../data/course_lead_scoring.csv.3'

     0K .......... .......... .......... .......... .......... 63% 1,33M 0s
    50K .......... .......... ........                        100% 2,61M=0,05s

2025-10-21 20:31:38 (1,63 MB/s) - '../data/course_lead_scoring.csv.3' saved [80876/80876]



In [3]:
df = pd.read_csv('./data/course_lead_scoring.csv')
df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,NaN,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259.0,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1
1460,referral,NaN,5,71016.0,self_employed,north_america,0,0.25,1


### Data preparation

* Check if the missing values are presented in the features.
* If there are missing values:
    * For categorical features, replace them with 'NA'
    * For numerical features, replace with with 0.0 

In [4]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [5]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [6]:
categorical_columns = df.select_dtypes(include=['object']).columns
df[categorical_columns] = df[categorical_columns].fillna('NA')

In [7]:
numeric_columns = df.select_dtypes(include=['number']).columns
df[numeric_columns] = df[numeric_columns].fillna(0.0)

In [8]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

### Question 1

What is the most frequent observation (mode) for the column `industry`?

- `NA`
- `technology`
- `healthcare`
- `retail`

#### **Answer:**

In [9]:
df.industry.mode()

0    retail
Name: industry, dtype: object

### Question 2

Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your dataset.
In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

- `interaction_count` and `lead_score`
- `number_of_courses_viewed` and `lead_score`
- `number_of_courses_viewed` and `interaction_count`
- `annual_income` and `interaction_count`

Only consider the pairs above when answering this question.


#### **Answer:**

In [10]:
corr_matrix = df[numeric_columns].corr().abs()

In [11]:
corr_matrix

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,0.023565,0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


annual_income and interaction_count

### Split the data

- Split your data in train/val/test sets with 60%/20%/20% distribution.
- Use Scikit-Learn for that (the `train_test_split` function) and set the seed to `42`.
- Make sure that the target value `converted` is not in your dataframe.

In [12]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [13]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [14]:
y_train = df_train.converted.values
y_test = df_test.converted.values
y_val = df_val.converted.values

del df_train['converted']
del df_test['converted']
del df_val['converted']

### Question 3

- Calculate the mutual information score between `converted` and other categorical variables in the dataset. Use the training set only.
- Round the scores to 2 decimals using `round(score, 2)`.

Which of these variables has the biggest mutual information score?

- `industry`
- `location`
- `lead_source`
- `employment_status`

#### **Answer:**

In [15]:
from sklearn.metrics import mutual_info_score

def mutual_info_converted(series):
    return mutual_info_score(series, y_train)

In [16]:
mi = df_train[categorical_columns].apply(mutual_info_converted)
mi.sort_values(ascending=False)

lead_source          0.035396
employment_status    0.012938
industry             0.011575
location             0.004464
dtype: float64

### Question 4

- Now let's train a logistic regression.
- Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
- Fit the model on the training dataset.
  - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
  - `model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)`
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

- 0.64
- 0.74
- 0.84
- 0.94

#### **Answer:**

In [17]:
df[categorical_columns].nunique()

lead_source          6
industry             8
employment_status    5
location             8
dtype: int64

In [18]:
categorical = list(df_train.select_dtypes(include=['object']).columns)
numeric = list(df_train.select_dtypes(include=['number']).columns)

In [19]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=False)
train_dicts = df_train[categorical + numeric ].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [20]:
val_dicts = df_val[categorical + numeric].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [21]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [22]:
y_pred = model.predict_proba(X_val)[:, 1]

In [23]:
converted_decision = (y_pred >= 0.5)

In [24]:
original_accuracy = (y_val == converted_decision).mean()

### Question 5

- Let's find the least useful feature using the _feature elimination_ technique.
- Train a model using the same features and parameters as in Q4 (without rounding).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

- `'industry'`
- `'employment_status'`
- `'lead_score'`

> **Note**: The difference doesn't have to be positive.

#### **Answer:** 
**obs.:** To simplify, I’ll exclude only the variables listed in the answer choices.

In [25]:
categorical, numeric

(['lead_source', 'industry', 'employment_status', 'location'],
 ['number_of_courses_viewed',
  'annual_income',
  'interaction_count',
  'lead_score'])

#### Training without 'industry':

In [26]:
categorical = ['lead_source', 'employment_status', 'location']

In [27]:
dv = DictVectorizer(sparse=False)
train_dicts = df_train[categorical + numeric ].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numeric].to_dict(orient='records')
X_val = dv.transform(val_dicts)

model_less_i = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model_less_i.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [28]:
y_pred = model_less_i.predict_proba(X_val)[:, 1]
converted_decision = (y_pred >= 0.5)
accuracy_less_i = (y_val == converted_decision).mean()

In [29]:
original_accuracy  - accuracy_less_i

np.float64(0.0)

#### Training without 'employment_status':

In [31]:
categorical = ['lead_source', 'industry', 'location']

In [32]:
dv = DictVectorizer(sparse=False)
train_dicts = df_train[categorical + numeric ].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numeric].to_dict(orient='records')
X_val = dv.transform(val_dicts)

model_less_es = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model_less_es.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [33]:
y_pred = model_less_es.predict_proba(X_val)[:, 1]
converted_decision = (y_pred >= 0.5)
accuracy_less_es = (y_val == converted_decision).mean()

In [34]:
original_accuracy  - accuracy_less_es

np.float64(0.0034129692832763903)

#### Training without 'lead_score'

In [35]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numeric = ['number_of_courses_viewed',
  'annual_income',
  'interaction_count']

In [36]:
dv = DictVectorizer(sparse=False)
train_dicts = df_train[categorical + numeric ].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numeric].to_dict(orient='records')
X_val = dv.transform(val_dicts)

model_less_ls = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model_less_ls.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [37]:
y_pred = model_less_ls.predict_proba(X_val)[:, 1]
converted_decision = (y_pred >= 0.5)
accuracy_less_ls = (y_val == converted_decision).mean()

In [38]:
original_accuracy  - accuracy_less_ls

np.float64(-0.0068259385665528916)

Which of following feature has the smallest difference? `'industry'`

### Question 6

- Now let's train a regularized logistic regression.
- Let's try the following values of the parameter `C`: `[0.01, 0.1, 1, 10, 100]`.
- Train models using all the features as in Q4.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these `C` leads to the best accuracy on the validation set?

- 0.01
- 0.1
- 1
- 10
- 100

> **Note**: If there are multiple options, select the smallest `C`.

#### **Answer:**

In [40]:
categorical = list(df_train.select_dtypes(include=['object']).columns)
numeric = list(df_train.select_dtypes(include=['number']).columns)

In [42]:
dv = DictVectorizer(sparse=False)
train_dicts = df_train[categorical + numeric ].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numeric].to_dict(orient='records')
X_val = dv.transform(val_dicts)

C_values = [0.01, 0.1, 1, 10, 100]
accuracies = {}

for C in C_values:
    model = LogisticRegression(
        solver='liblinear',
        C=C,
        max_iter=1000,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    converted_decision = (y_pred >= 0.5)

    accuracy = (y_val == converted_decision).mean()
    accuracies[C] = round(accuracy, 3)

In [43]:
accuracies

{0.01: np.float64(0.7),
 0.1: np.float64(0.7),
 1: np.float64(0.7),
 10: np.float64(0.7),
 100: np.float64(0.7)}

Which of these `C` leads to the best accuracy on the validation set?

- 0.01
- 0.1
- 1
- 10
- 100

> **Note**: If there are multiple options, select the smallest `C`.:

**0.01**